In [1]:
import ephem
import math
import numpy as np
from datetime import datetime

In [2]:
N = 1
S = -1
E = 1
W = -1

In [3]:
# Mind blown... python uses Bakers Rounding.  Got to resort to this to get proper rounding.
from decimal import *
getcontext().prec = 4
getcontext().rounding = ROUND_HALF_UP

def normal_round(value, precision):
    value = value * 10**precision
    value = Decimal(value).to_integral_value()
    value = value / 10**precision
    return float(value)

In [4]:
def createAngle(degrees, minutes, sign):
    if (sign == None): 
        sign = 1
    return (degrees + minutes/60.0) * sign

def toDegreesAndMinutes(angle):
    sign = 1
    if (angle < 0): sign = -1
    angle = angle*sign
    degrees = math.floor(angle)
    minutes = normal_round((angle - degrees) * 60.0, 2)
    
    return degrees*sign, minutes
    
def angleToString(angle):
    degrees, minutes = toDegreesAndMinutes(angle)
    return "{} degrees; {} minutes".format(degrees, minutes)

def angleToStringDelta(angle):
    degrees, minutes = toDegreesAndMinutes(angle)
    if (degrees == 0):
        return "{} minutes".format(minutes)
    else:
        return "{} degrees; {} minutes".format(degrees, minutes)
    
def diffAngle (angle1, angle2, zero_threshold):
    threesixty_threshold = 360-zero_threshold
    
    if ((angle1 < zero_threshold) and (angle2 > threesixty_threshold)):
        angle1 += 360 # Move up Angle 1
    elif((angle2 < zero_threshold) and (angle1 > threesixty_threshold)):
        angle2 += 360 # Move up Angle 2
    
    return abs (angle1-angle2)
    

In [5]:
def compute_GHA_dec(celestial_object, utc):
    gmt_long = '0:0:0'          #  deg, min, sec
    myloc_date = utc
    # observer for greenwich  gst
    utcz = ephem.Observer()
    utcz.date = myloc_date
    utcz.long = gmt_long
    gha_aries = math.degrees(utcz.sidereal_time())
    
    celestial_object.compute(utcz)
    sha = 360.0-math.degrees(celestial_object.ra)
    gha = sha + gha_aries
    
    if (gha > 360.0):
        gha = gha-360.0
    
    return gha, math.degrees(celestial_object.dec)

In [6]:
def compute_LHA(GHA, dr_lon):
    # Compute LHA.
    if (dr_lon < 0):
        # Western hemisphere.  LHA = GHA - a_lon
        LHA = GHA - (dr_lon*W)
    else:
        # Easter hemisphere.  LHA = GHA + a_lon
        LHA = GHA + (dr_lon*E)
        
    if (LHA < 0):
        LHA += 360.0
    elif (LHA > 360.0):
        LHA -= 360.0
    return LHA

In [7]:
def sight_reduction(dec, lat, LHA):
    dec_rads = math.radians(dec)
    lat_rads = math.radians(lat)
    LHA_rads = math.radians(LHA)
    
    if ((dec_rads < 0) and (lat_rads < 0)):
        # If dec and lat are same hemisphere (i.e. SAME), then both values should be positive.
        # here there are both south (so negative) take the absolute value so that they are both positive.
        dec_rads = abs(dec_rads)
        lat_rads = abs(lat_rads)
    elif ((dec_rads > 0) and (lat_rads < 0)):
        # if dec and lat different hemispheres (i.e. CONTRARY), then dec should be made negative.
        # here we have a south latitude and north declination, so shift the negative to dec.
        dec_rads = dec_rads *-1
        lat_rads = lat_rads *-1
    
    # Hc
    sin_Hc_rads = math.sin(lat_rads)*math.sin(dec_rads) + math.cos(lat_rads)*math.cos(dec_rads)*math.cos(LHA_rads) 
    Hc_rads = math.asin(sin_Hc_rads)
    Hc = math.degrees(Hc_rads)
    
    #Z       
    cos_Z_rads_num = math.sin(dec_rads) - math.sin(Hc_rads)*math.sin(lat_rads)
    cos_Z_rads_den = math.cos(Hc_rads) * math.cos(lat_rads) 
    cos_Z_rads = cos_Z_rads_num / cos_Z_rads_den
    
    # Clip values that are > 1 or < -1  (usually introduced due to rounding error)
    if (cos_Z_rads > 1):
        print("cos_Z_rads > 1 {}".format(cos_Z_rads))
        cos_Z_rads = 1
    elif (cos_Z_rads < -1):
        print("cos_Z_rads < 1 {}".format(cos_Z_rads))
        cos_Z_rads = -1
    
    Z = math.degrees(math.acos(cos_Z_rads))
    
    return Hc, Z

In [8]:
Off = 1
On = -1

LowerLimb = 0
UpperLimb = 1

def compute_refraction_corr(Ha):
    corr_min = 1.0/ math.tan(math.radians(Ha + (7.31/(Ha+4.4))))
    corr_min = normal_round(corr_min, 1)
    #print("refraction: {}".format(corr_min))
    return createAngle(0, corr_min, -1)

def compute_sun_semi_diameter(utc):
    date_jan_1_year = datetime.strptime("{}/1/1 0:0:0".format(utc.year), "%Y/%m/%d %H:%M:%S")
    days = (utc - date_jan_1_year).days
    if (days >= 0 and days < 14):
        sd = 16.3
    elif (days >= 14 and days < 54):
        sd = 16.2
    elif (days >=54 and days < 79):
        sd = 16.1
    elif (days >= 79 and days < 100):
        sd = 16.0
    elif (days >=100 and days < 124):
        sd = 15.9
    elif (days >= 124 and days < 157):
        sd = 15.8
    elif (days >= 157 and days < 214):
        sd = 15.7
    elif (days >= 214 and days < 247):
        sd = 15.8
    elif (days >= 247 and days < 271):
        sd = 15.9
    elif (days >= 271 and days < 293):
        sd = 16.0
    elif (days >= 293 and days <317):
        sd = 16.1
    elif (days >= 317 and days < 350):
        sd = 16.2
    else:
        sd = 16.3
        
    return sd

def sun_alt_corr(Ha, limb, utc):
    sd = compute_sun_semi_diameter(utc)
    if (limb == UpperLimb):
        sd=sd*-1
    
    # HP is 0.1 if Ha < 70 degrees and 20 minutes  Found from Almanic Web App.
    if (Ha < createAngle(70, 20, None)):
        sd=sd+0.1
            
    sd = createAngle(0, sd, None)
    corr = sd + compute_refraction_corr(Ha)

    return corr

def compute_dip(heightEyeFt):
    dip = 0.97*math.sqrt(heightEyeFt)
    return normal_round(dip, 1)


In [9]:
def compute_Zn(Z, lat, LHA):
    #Zn
    Zn = Z
    if (lat > 0):
        # Northern latitude.
        if (LHA < 180):
            Zn = 360 - Z
    else:
        # Souther latitude.
        if (LHA > 180):
            Zn = 180-Z
        else:
            Zn = 180+Z
            
    if (Zn == 360):
        Zn = 0
    return Zn

In [10]:
def compute_a(Ho, Hc):
    if (Ho > Hc):
        return Ho - Hc, 'T'
    if (Ho < Hc):
        return Hc - Ho, 'A'

In [11]:
def compute_lat_from_Ho(dec, Ho, dr_lat):
    looking = N
    if (dr_lat > 23.5):
        looking = S
    elif (dr_lat < -23.5):
        looking = N
    elif (dec < dr_lat):
        looking = S
    else:
        looking = N
    
    zenith = 90 - Ho
    return dec - zenith*looking

In [12]:
def compute_position(celestial_obj_string, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt):
    print("utc: {}".format(utc))
    print("dr_lat: {}".format(angleToString(dr_lat)))
    print("dr_lon: {}".format(angleToString(dr_lon)))
    
    dt = datetime.strptime(utc, '%Y/%m/%d %H:%M:%S')
    celestial_obj = None
    
    alt_correction = 0
    
    print("Hs: {}".format(angleToString(Hs)))
    print("ic: {}".format(angleToString(ic)))
    print("heightEyeFt: {}".format(heightEyeFt))
    
    dip = compute_dip(heightEyeFt) 
    print("dip: {}".format(dip))    
    
    Ha = Hs + ic - createAngle(0, dip, None)
    print("Ha: {}".format(angleToString(Ha)))
    
    if (celestial_obj_string == "Sun-LL"):
        celestial_obj = ephem.Sun()
        alt_correction = sun_alt_corr(Ha, LowerLimb, dt)
    elif (celestial_obj_string == "Sun-UL"):
        celestial_obj = ephem.Sun()
        alt_correction = sun_alt_corr(Ha, UpperLimb, dt)
    else:
        celestial_obj = ephem.star(celestial_obj_string)
        alt_correction = compute_refraction_corr(Ha)
    
    Ho = Ha + alt_correction
    print("alt_correction: {}".format(angleToString(alt_correction)))
    print("Ho: {}".format(angleToString(Ho)))

    GHA, dec = compute_GHA_dec(celestial_obj, utc)
    #print("GHA: {}".format(angleToString(GHA)))
    #print("dec: {}".format(angleToString(dec)))

    LHA = compute_LHA(dr_lon, GHA)
    print("LHA: {}".format(angleToString(LHA)))

    Hc, Z = sight_reduction(dec, dr_lat, LHA)
    Zn = compute_Zn(Z, dr_lat, LHA)
    print("Hc: {}".format(angleToString(Hc)))
    print("Z: {}".format(angleToString(Z)))
    print("Zn: {}".format(angleToString(Zn)))

    a, ta = compute_a(Ho, Hc)
    print("a: {} {} {}".format(angleToString(a), ta, Zn))
    
    return Ho, LHA, Hc, Zn, a, ta

In [13]:
data = [
    ["Sun-LL", "1978/7/25 23:4:0", createAngle(56, 2, N), createAngle(164, 30, W), createAngle(54, 5, None), createAngle(0, 2, On), 9],
    ["Sun-UL", "1978/10/27 22:49:0", createAngle(10, 0, S), createAngle(166, 15, W), createAngle(87, 20, None), createAngle(0, 2, Off), 25],
    ["Sun-LL", "1978/10/25 6:48:0", createAngle(35, 54, N), createAngle(74, 2, E), createAngle(41, 30.5, None), createAngle(0, 4, Off), 10],
    ["Sun-LL", "1981/3/27 13:7:0", createAngle(37, 40, N), createAngle(15, 24, W), createAngle(54, 41.3, None), createAngle(0, 2, Off), 9],
    ["Sun-LL", "1981/3/28 7:48:0", createAngle(22, 38, N), createAngle(64, 17, E), createAngle(71, 9.8, None), createAngle(0, 2.2, On), 9],
    ["Sun-LL", "1978/10/26 0:51:0", createAngle(27, 6, S), createAngle(163, 16, E), createAngle(74, 59.8, None), createAngle(0, 1.4, Off), 9],
    ["Sun-UL", "1978/7/24 17:48:0", createAngle(28, 44, S), createAngle(85, 17, W), createAngle(42, 51.2, None), createAngle(0, 1.5, On), 9],
    ["Sun-LL", "1978/7/25 20:50:0", createAngle(44, 10, N), createAngle(131, 0, W), createAngle(65, 22.5, None), createAngle(0, 2, On), 9],
    ["Sun-LL", "1981/3/28 21:51:0", createAngle(16, 40, S), createAngle(146, 30, W), createAngle(69, 44.2, None), createAngle(0, 2.5, Off), 12],
    ["Sun-LL", "1978/10/25 10:4:0", createAngle(34, 29, N), createAngle(25, 0, E), createAngle(43, 11.7, None), createAngle(0, 1.5, Off), 14],
    
    ["Altair", "1978/7/26 4:51:30", createAngle(45, 30, N), createAngle(126, 27, W), createAngle(35, 18.9, None), createAngle(0, 0, Off), 12],
    ["Arcturus", "1978/7/25 3:49:3", createAngle(45, 30, N), createAngle(120, 58, W), createAngle(57, 57.4, None), createAngle(0, 1.8, On), 16],
    ["Altair", "1978/7/25 4:7:22", createAngle(44, 36, N), createAngle(122, 14, W), createAngle(30, 35.4, None), createAngle(0, 2, Off), 16],
    ["Antares", "1978/7/25 4:7:22", createAngle(44, 36, N), createAngle(122, 14, W), createAngle(18, 54.3, None), createAngle(0, 2, Off), 16],
    ["Arcturus", "1978/7/26 4:48:17", createAngle(45, 30, N), createAngle(126, 27, W), createAngle(50, 50.9, None), createAngle(0, 0, Off), 12],
    ["Regulus", "1981/3/28 3:49:5", createAngle(45, 21, N), createAngle(130, 3, W), createAngle(42, 58.5, None), createAngle(0, 1.2, Off), 9]
]

expected = [
    [createAngle(359, 53.3, None), createAngle(54, 15.2, None), createAngle(53, 32.4, None), createAngle(0, 42.8, None), "T", 179.8],
    [createAngle(0, 1.6, None), createAngle(87, 1, None), createAngle(87, 5.5, None), createAngle(0, 4.6, None), "A", 180.5],
    [createAngle(359, 59.5, None), createAngle(41, 46.5, None), createAngle(42, 6.2, None), createAngle(0, 19.7, None), "A", 180],
    [createAngle(0, 0.3, None), createAngle(54, 55.8, None), createAngle(55, 1.4, None), createAngle(0, 5.6, None), "A", 180],
    [createAngle(359, 59.9, None), createAngle(71, 20.4, None), createAngle(70, 21.6, None), createAngle(0, 58.8, None), "T", 180],
    [createAngle(359, 59.8, None), createAngle(75, 14.1, None), createAngle(75, 9.3, None), createAngle(0, 4.8, None), "T", 0],
    [createAngle(0, 6.5, None), createAngle(42, 30.1, None), createAngle(41, 26, None), createAngle(1, 4.1, None), "T", 359.9],
    [createAngle(359, 53.3, None), createAngle(65, 32.9, None), createAngle(65, 25.6, None), createAngle(0, 7.4, None), "T", 179.7],
    [createAngle(0, 0.6, None), createAngle(69, 59.0, None), createAngle(70, 6.7, None), createAngle(0, 7.6, None), "A", 0],
    [createAngle(359, 57.8, None), createAngle(43, 24.7, None), createAngle(43, 28.4, None), createAngle(0, 3.7, None), "A", 180],   
    
    [createAngle(312, 31.1, None), createAngle(35, 14.1, None), createAngle(35, 16.1, None), createAngle(0, 2, None), "A", 116.9],
    [createAngle(25, 7.9, None), createAngle(57, 51.1, None), createAngle(56, 34.6, None), createAngle(1, 16.5, None), "T", 226.7],
    [createAngle(304, 41.2, None), createAngle(30, 31.8, None), createAngle(30, 31.9, None), createAngle(0, .1, None), "A", 109.4],
    [createAngle(355, 6, None), createAngle(18, 49.5, None), createAngle(18, 52.5, None), createAngle(0, 3, None), "A", 175.4],
    [createAngle(35, 29, None), createAngle(50, 46.7, None), createAngle(50, 45.0, None), createAngle(0, 1.7, None), "T", 240],
    [createAngle(320, 51.4, None), createAngle(42, 55.7, None), createAngle(42, 58.3, None), createAngle(0, 2.6, None), "A", 122.5]
]

threshold = createAngle(0, 0.3, None)
Zn_threshold = createAngle(1, 0, None)  # Answers from book use a method of rounding to nearest degree.

success = True

for i in range (len(data)):
    celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt = data[i]
    Ho, LHA, Hc, Zn, a, ta = compute_position (celestial_object, utc, dr_lat, dr_lon, Hs, ic, heightEyeFt)
    LHA_expected, Ho_expected, Hc_expected, a_expected, ta_expected, Zn_expected = expected[i]
    
    print ("SIGHTING for {} at {}".format(celestial_object, utc))
    
    LHA_delta = diffAngle(LHA, LHA_expected, threshold)
    LHA_passed = (LHA_delta<threshold)
    print ("{}: LHA={}; delta={}".format("PASSED" if LHA_passed else "FAILED", angleToString(LHA), angleToStringDelta(LHA_delta)))
    
    Ho_delta = abs(Ho-Ho_expected)
    Ho_passed = (Ho_delta<threshold)
    print ("{}: Ho={}; delta={}".format("PASSED" if Ho_passed else "FAILED", angleToString(Ho), angleToStringDelta(Ho_delta)))
    
    Hc_delta = abs(Hc-Hc_expected)
    Hc_passed = (Hc_delta<threshold)
    print ("{}: Hc={}; delta={}".format("PASSED" if Hc_passed else "FAILED", angleToString(Hc), angleToStringDelta(Hc_delta)))
    
    a_delta = abs(a-a_expected)
    a_passed = ((a_delta<threshold) and (ta == ta_expected))
    print ("{}: a={} {}; delta={}".format("PASSED" if a_passed else "FAILED", angleToString(a), ta, angleToStringDelta(a_delta)))
    if (ta != ta_expected):
        print("MISMATCH on T/A:  ta={}; ta_expected={}".format(ta, ta_expected))
   
    Zn_delta = diffAngle(Zn, Zn_expected, Zn_threshold)
    Zn_passed = (Zn_delta<Zn_threshold)
    print ("{}: Zn={}; delta={}".format("PASSED" if Zn_passed else "FAILED", angleToString(Zn), angleToStringDelta(Zn_delta)))
    
    print ("==========================")
    
    if (success == True):
        success = LHA_passed and Ho_passed and Hc_passed and a_passed and Zn_passed

    
print("Overall result: {}".format("PASSED" if (success==True) else "FAILED! Check above results."))



utc: 1978/7/25 23:4:0
dr_lat: 56 degrees; 2.0 minutes
dr_lon: -164 degrees; 30.0 minutes
Hs: 54 degrees; 5.0 minutes
ic: 0 degrees; 2.0 minutes
heightEyeFt: 9
dip: 2.9
Ha: 54 degrees; 0.1 minutes
alt_correction: 0 degrees; 15.1 minutes
Ho: 54 degrees; 15.2 minutes
LHA: 359 degrees; 53.37 minutes
Hc: 53 degrees; 32.31 minutes
Z: 179 degrees; 49.49 minutes
Zn: 179 degrees; 49.49 minutes
a: 0 degrees; 42.89 minutes T 179.82478630744768
SIGHTING for Sun-LL at 1978/7/25 23:4:0
PASSED: LHA=359 degrees; 53.37 minutes; delta=0.07 minutes
PASSED: Ho=54 degrees; 15.2 minutes; delta=0.0 minutes
PASSED: Hc=53 degrees; 32.31 minutes; delta=0.09 minutes
PASSED: a=0 degrees; 42.89 minutes T; delta=0.09 minutes
PASSED: Zn=179 degrees; 49.49 minutes; delta=1.49 minutes
utc: 1978/10/27 22:49:0
dr_lat: -10 degrees; 0.0 minutes
dr_lon: -166 degrees; 15.0 minutes
Hs: 87 degrees; 20.0 minutes
ic: 0 degrees; 2.0 minutes
heightEyeFt: 25
dip: 4.9
Ha: 87 degrees; 17.1 minutes
alt_correction: 0 degrees; 16.1 min